In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('inputs/data_storeproduct_202208111119.csv', usecols = ['id', 'name'])

In [3]:
df.shape

(820512, 2)

In [4]:
df.drop_duplicates(subset = 'name', inplace = True)

In [5]:
df.shape

(306052, 2)

In [6]:
df['name'] = df['name'].str.upper()

In [7]:
df['clean'] = df['name']

In [8]:
def remove_pairs(name, pattern):
    words = name.split()

    for k, v in enumerate(words):
        if v == pattern:
            try:
                words.remove(words[k + 1])
                words.remove(v)
            except:
                pass
                
    return ' '.join(words)

In [9]:
patterns = ['REF', 'FPS', 'C/', 'S/']

In [10]:
for i in patterns:
    df['clean'] = df['clean'].apply(lambda x: remove_pairs(x, i))

In [11]:
def remove_metrics(name):
    to_remove = []
    
    for word in name.split():
        if word[0].isdigit() and 'X' in word and word[-1] == 'M' :
            to_remove.append(word)

    if len(to_remove) > 0:
        return ''. join([name.replace(x, '') for x in to_remove])
    
    return name

In [12]:
df['clean'] = df['clean'].apply(lambda x: remove_metrics(x))

In [13]:
def remove_startswith(pattern):
    return lambda words: words.replace(''.join([x for x in words.split() if x.startswith(pattern)]), '')

In [14]:
patterns = ['C/', 'S/', 'FPS']

In [15]:
for i in patterns:
    df['clean'] = df['clean'].apply(remove_startswith(i))

In [16]:
def remove_endswith(pattern, numbers):
    return lambda words: words.replace(''.join([x for x in words.split(' ') if x.endswith(pattern) and x[:numbers].isdigit()]), '')

In [17]:
endswith_ = {
    'G':      -len('G'),
    'L':      -len('L'),
    'ML':     -len('ML'),
    'UN':     -len('UN'),
    'GR':     -len('GR'),
    'CPS':    -len('CPS'),
    'VOL':    -len('VOL'),
    'CAP':    -len('CAP'),
    'UNID':   -len('UNID'),
    'UNID.':  -len('UNID.'),
}

In [18]:
# for k, v in endswith_.items():
#     df['cleaned'] = df['cleaned'].apply(remove_endswith(k, v))

In [19]:
df['clean'] = df['clean'].str.strip()

In [20]:
df['num_words'] = df['clean'].apply(lambda x: len(str(x).split()))

In [63]:
df[df['id'].isin([61903, 212540, 62840, 208009, 894356, 381777, 458513, 375785, 210052, 347881

])]

,id,name,clean,num_words
8,458513,CHOC TWIX 15GR,CHOC TWIX 15GR,3
65,375785,COENZIMA Q10 500MG 60CPS,COENZIMA Q10 500MG 60CPS,4
3810,208009,BAND AID VARIADOS C 30UNID,BAND AID VARIADOS C 30UNID,5
4262,210052,AGUA OX.BEIRA ALTA 20VOL,AGUA OX.BEIRA ALTA 20VOL,4
29601,894356,BAND AID SKIN FLEX 25UNID.,BAND AID SKIN FLEX 25UNID.,5
54775,347881,MULTIGRIP BLIST 10CAP,MULTIGRIP BLIST 10CAP,3
210436,212540,FRESUBIN HP ENERGY 1000ML,FRESUBIN HP ENERGY 1000ML,4
211906,61903,LAVIE ALCOOL GEL 70 460G,LAVIE ALCOOL GEL 70 460G,5
213106,62840,ENERGETICO GFA 1L 6UN,ENERGETICO GFA 1L 6UN,4
816883,381777,AGUA PERFUMADA WOOD 1L,AGUA PERFUMADA WOOD 1L,4


In [62]:
df[df['name'].str.endswith('0CAP')]

,id,name,clean,num_words
53136,418510,SUPRAVIT MEMORIA C/60CAP,SUPRAVIT MEMORIA,2
54775,347881,MULTIGRIP BLIST 10CAP,MULTIGRIP BLIST 10CAP,3
55047,1022551,COLAGENO TIPO II 40MG 60CAP,COLAGENO TIPO II 40MG 60CAP,5
55179,1021825,RESFEGRIPE 20CAP,RESFEGRIPE 20CAP,2
57474,283703,MACRODANTINA 100MG C/40CAP,MACRODANTINA 100MG,2
...,...,...,...,...
743031,426335,(*)7366701-TIAMINA + MAGNESIO QUELATO - 30CAP,(*)7366701-TIAMINA + MAGNESIO QUELATO - 30CAP,6
786747,479075,VARINATI 250MG 30CAP,VARINATI 250MG 30CAP,3
797808,371605,ALCACHOFRA 350MG 60CAP,ALCACHOFRA 350MG 60CAP,3
798985,893751,INPRUV DK 1.000UI+100MCG C 30CAP,INPRUV DK 1.000UI+100MCG C 30CAP,5


In [31]:
df.loc[[
    10, 1371, 1526, 177283, # remove_pairs()
    3542, 210239, # remove_metrics() 
    12, 2018, 208178, # remove_startswith()
    210435, # remove_endswith()
]]

,id,name,clean,num_words
10,458515,BITUFO - PASSA FIO WAY C/ 30,BITUFO - PASSA FIO WAY,5
1371,355721,NOPLAK S/ ALCOOL 250ML,NOPLAK 250ML,2
1526,356920,KIT MAMADEIRA COLORS BICO RED REF 926 (CEFISA),KIT MAMADEIRA COLORS BICO RED (CEFISA),6
177283,56645,AVENE EMULSAO SOLAR TS FPS 70 40G,AVENE EMULSAO SOLAR TS 40G,5
3542,460190,"ESP MIC CIEX 2,5CMX4,5M",ESP MIC CIEX,3
210239,268953,"ESPARADRAPO 25MMX4,5MM NEXCARE",ESPARADRAPO NEXCARE,2
12,458516,ISABABY - HAST FLEX BABY C/50,ISABABY - HAST FLEX BABY,5
2018,357133,TENYS PE BARUEL AEROSOL S/PERFUME 110G,TENYS PE BARUEL AEROSOL 110G,5
208178,311398,EPISOL HOMEM FPS45 60G,EPISOL HOMEM 60G,3
210435,212052,CAMOMILA CHA 20G FLORIEN,CAMOMILA CHA 20G FLORIEN,4


In [23]:
# df.to_csv('outputs/clean.csv', index = False)

---

In [24]:
# def remove_parenthesis(name):
#     pass

In [25]:
# Criar coluna indicando quais produtos tiveram o nome modificado

To replace:

In [26]:
# dict_ = {
#     'ABS': 'ABSORVENTE',
#     'ESP': 'ESPARADRAPO',
#     'SAB': 'SABONETE',
#     'DES': 'DESODORANTE',
#     'ESC': 'ESCOVA',
#     'LOC': 'LOCAO',
#     'TRAD': 'TRADICIONAL',
#     'CURAT': 'CURATIVO',
# }

In [27]:
# unique_ = ['ML', 'PET']